In [1]:
!pip install gradio huggingface_hub -q

In [ ]:
import gradio as gr
from huggingface_hub import InferenceClient
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline
import json
from google.colab import userdata # Import userdata to access Colab secrets

# Define the directory where the model and associated files are saved
# Changed from Google Drive path to a local Colab path
output_dir = '/content/model'

# Load the tokenizer from the specified directory
tokenizer = AutoTokenizer.from_pretrained(output_dir, use_fast=False)

# Load label mappings from JSON files
with open(f"{output_dir}/label_map.json", "r") as f:
    label_map = json.load(f)
with open(f"{output_dir}/reverse_label_map.json", "r") as f:
    reverse_label_map = {int(k): v for k, v in json.load(f).items()} # Convert keys to int

# Determine the number of labels for model configuration
num_labels = len(label_map)

# Load the fine-tuned sequence classification model
model = AutoModelForSequenceClassification.from_pretrained(output_dir, num_labels=num_labels)

# Move the model to the appropriate device (GPU if available, else CPU)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

# Initialize pre-trained sentiment and emotion analysis pipelines
sentiment_analyzer = pipeline('sentiment-analysis', model='cardiffnlp/twitter-roberta-base-sentiment-latest', device=0 if torch.cuda.is_available() else -1)
emotion_analyzer = pipeline('text-classification', model='SamLowe/roberta-base-go_emotions', top_k=None, device=0 if torch.cuda.is_available() else -1)


def perform_all_analyses(input_text):
    """
    Performs mental health classification, sentiment analysis, and emotion analysis
    on a given input text.
    """
    # Tokenize input for the mental health classification model
    encoded_input = tokenizer(input_text, padding=True, truncation=True, max_length=512, return_tensors='pt')

    # Move inputs to the designated device
    input_ids = encoded_input['input_ids'].to(device)
    attention_mask = encoded_input['attention_mask'].to(device)

    # Perform mental health prediction
    model.eval()
    with torch.no_grad():
        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        prediction = torch.argmax(logits, axis=-1).item()
    mental_health_label = reverse_label_map[prediction]

    # Perform sentiment analysis
    sentiment_result = sentiment_analyzer(input_text)[0]

    # Perform emotion analysis and get top 3 emotions
    emotion_raw_results = emotion_analyzer(input_text)[0]
    sorted_emotions = sorted(emotion_raw_results, key=lambda x: x['score'], reverse=True)
    top_emotions = sorted_emotions[:3]

    # Return all analysis results
    return {
        'mental_health_prediction': mental_health_label,
        'sentiment_analysis': {
            'label': sentiment_result['label'],
            'score': sentiment_result['score']
        },
        'emotion_analysis': top_emotions
    }

# Setup the Generative Model (The "Voice" of the bot)
# Retrieve the token from Colab secrets. Make sure you've saved your Hugging Face token under 'HF_TOKEN'.
HUGGING_FACE_TOKEN = userdata.get('HF_TOKEN')
client = InferenceClient("HuggingFaceH4/zephyr-7b-beta", token=HUGGING_FACE_TOKEN)

def generate_smart_response(user_text, analysis_result):
    """
    Generates a human-like, empathetic response based on analysis results.
    """
    status = analysis_result['mental_health_prediction']
    sentiment = analysis_result['sentiment_analysis']['label']
    primary_emotion = analysis_result['emotion_analysis'][0]['label']
    top_emotions_str = ", ".join([f"{e['label']} (score: {e['score']:.2f})" for e in analysis_result['emotion_analysis'][:3]])

    # Conditional guidance for the LLM based on perceived severity
    severity_guidance = ""
    if status in ['depression', 'anxiety', 'burnout', 'stress']:
        severity_guidance = (
            "Given the nature of their feelings, gently suggest exploring ways to support their well-being, "
            "such as talking to someone they trust, engaging in self-care, or seeking professional guidance. "
            "Integrate this suggestion naturally into your response."+ # Concatenated string for better formatting
            "Keep the response short (under 3 sentences) and supportive."
        )

    # Construct the prompt for the LLM
    system_prompt = (
        f"You are a compassionate mental health support assistant. "
        f"The user just said: '{user_text}'. "
        f"Based on an internal analysis, I've identified: "
        f"- Mental state: '{status}'\n"
        f"- Overall sentiment: '{sentiment}'\n"
        f"- Top emotions: {top_emotions_str}. "
        f"Please respond in a warm, validating, and human-like way. "
        f"First, empathetically acknowledge these insights (mental state, sentiment, and primary emotion) directly. "
        f"Then, offer supportive words and a practical suggestion relevant to their mental state. "
        f"Keep your response concise, around 2-4 sentences, combining acknowledgment and support naturally. "
        f"{severity_guidance}"
    )

    messages = [{"role": "system", "content": system_prompt},
                {"role": "user", "content": user_text}]

    # Generate response using the inference client
    response = ""
    for message in client.chat_completion(messages, max_tokens=150, stream=True):
        token = message.choices[0].delta.content
        response += token

    return response

def chat_logic(message, history):
    """
    Main chat logic: analyzes user input and generates a bot response.
    """
    # Run backend analysis on the user message
    raw_analysis = perform_all_analyses(message)

    # Generate a human-like response
    bot_response = generate_smart_response(message, raw_analysis)

    return bot_response

with gr.Blocks(theme=gr.themes.Soft()) as demo:
    gr.Markdown("# Mental Health Companion")
    gr.Markdown("I'm here to listen. Tell me how you are feeling today.")

    # Initialize the chatbot component with an initial bot message
    chatbot = gr.Chatbot(
        value=[{"role": "assistant", "content": "Hello. I'm here to support you. How are you feeling right now?"}],
        height=400,
        type="messages"
    )

    msg = gr.Textbox(placeholder="Type your feelings here...", show_label=False)
    clear = gr.ClearButton([msg, chatbot])

    def user_interaction(user_message, history):
        # Append user message to history and clear input box
        history.append({"role": "user", "content": user_message})
        return "", history

    def bot_interaction(history):
        # Get the last user message from history
        user_message = history[-1]['content']

        # Get bot response
        response_text = chat_logic(user_message, history)

        # Append bot response to history
        history.append({"role": "assistant", "content": response_text})
        return history

    # Configure chat interaction flow
    msg.submit(user_interaction, [msg, chatbot], [msg, chatbot], queue=False).then(
        bot_interaction, [chatbot], [chatbot]
    )

# Launch the Gradio application
demo.launch(share=True, debug=True)

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use cuda:0
Device set to use cuda:0
/tmp/ipython-input-2456631038.py:135: DeprecationWarning: The 'theme' parameter in the Blocks constructor will be removed in Gradio 6.0. You will need to pass 'theme' to Blocks.launch() instead.
  with gr.Blocks(theme=gr.themes.Soft())

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://7631f7908f01c65a54.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
